In [49]:
#Install and import dependencies
import pandas as pd 
import numpy as np
%pip install transformers 

Note: you may need to restart the kernel to use updated packages.


In [31]:
#BERT is the sentiment model (1-5 sentiment score)
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests 
from bs4 import BeautifulSoup
import re 
#ArgMax for torch, takes the highest sequence request
#'re' is the regex function to extract specific function

In [32]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
#Creating our tokenizer, using the pretrained model
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [33]:
tokens = tokenizer.encode('I hated this, absolutely the worst' , return_tensors='pt')
print(tokens)
#Encoding our string turning it into a list of integers
#We can also decode, making sure to use it as a list
tokenizer.decode(tokens[0])
#Grabs internal string by using [0]

tensor([[  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 10103, 43060,
           102]])


'[CLS] i hated this, absolutely the worst [SEP]'

In [34]:
#Find the result using our model
result = model(tokens)
print(result)
torch.argmax(result.logits)
#Finding the highest value, what position represents the actual sentiment

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


tensor(0)

In [35]:
int(torch.argmax(result.logits))+1
#Returning a sentiment rating of 1, the lowest

1

***Let's try to collect some reviews now***

In [47]:
r = requests.get('https://www.yelp.com/biz/crisp-and-green-vernon-hills')
#Requests and grabs the webpage | r.text gets the entire webpage
soup = BeautifulSoup(r.text, 'html.parser')
#BS strains the text, setting an html parser
regex = re.compile('.*comment.*')
#Regex extracts and compiles the specific comment within the class 
results = soup.find_all('p' , {'class' : regex})
#We're going through our soup to find all tags which match the specific formatting 
reviews = [result.text for result in results]
reviews

["Crisp & Green is located in Mellody Farms. We came by for dinner one evening after shopping in the area and we were looking for something light but filling, and naturally Crisp & Green was the perfect option. We came in just before dinner time and there were some folks there but as we looked through the menu, a bunch of folks came in for both carry out and to dine in. So great to know that if I'm in a rush, I can order ahead and just come in and pick it up. I opted to build my own bowl, it was my first time ordered a non-curated bowl so the process was a bit different as the order of what I could pick from wasn't really in front of me but I ended up with a salad that I really enjoyed. From the bowl itself it didn't look like much but once you got into it, boy does it fill you up. Overall, we had a great time, the staff was super helpful and patient and I had an amazing time enjoying dinner with my bjs and in a bright area with great windows to look outside. Also their drink options w

In [53]:
df = pd.DataFrame(np.array(reviews), columns = ['review'] )
#Converting reviews to numPY arrary 
print(f"This is our dataframe {df}")

This is our dataframe                                                review
0   Crisp & Green is located in Mellody Farms. We ...
1   Hello Kesha! We are so happy you had an amazin...
2   DO NOT USE THE APP!!!! Literally waited over a...
3   Hey Laura! We truly apologize for the technica...
4   5 stars for Crisp & Green.Crisp & Green is a q...
5   Hi there Frank! Thank you very much for coming...
6   My first time at Crisp & Green and I have to s...
7   Hey there Steve! Thank you so much for the com...
8   Very upset  we ordered here before and love it...
9   You are too good to us, Ashley! What an honor ...
10  My husband and I had a wonderful first experie...
11  Thank you for the 5 stars, Sandy! We're beyond...
12  I crave their grain/salad bowls like once a we...
13  Hey Anne! We're thrilled to hear you checked u...
14  This is the kind of place I'm looking for. War...
15  We're so happy you had a 5-star time with us, ...
16  I don't usually eat salads but in an attempt t...
17  JR

In [55]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors = 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1


In [58]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
#Can only handle 512 tokens at once, so we only grab the first 512 tokens
df

,review,sentiment
0,Crisp & Green is located in Mellody Farms. We ...,4
1,Hello Kesha! We are so happy you had an amazin...,5
2,DO NOT USE THE APP!!!! Literally waited over a...,1
3,Hey Laura! We truly apologize for the technica...,5
4,5 stars for Crisp & Green.Crisp & Green is a q...,5
5,Hi there Frank! Thank you very much for coming...,5
6,My first time at Crisp & Green and I have to s...,4
7,Hey there Steve! Thank you so much for the com...,5
8,Very upset we ordered here before and love it...,1
9,"You are too good to us, Ashley! What an honor ...",5


In [61]:
def is_not_owner_response(tag):
    if tag.has_attr('class'):
        class_list = tag['class']
        for class_name in class_list:
            if 'truncated__09f24__lSBbT' in class_name:
                return False
    return True


# Find all review containers (inspect the HTML to find the correct class)
review_containers = soup.find_all('div', {'class': re.compile(r'review__\d+')})  # Adjust the regex to match review containers

# Extract review text from each container while filtering out owner responsesreviews = []
for container in review_containers:
    review_text = container.find('p')
    if review_text and is_not_owner_response(container):
        reviews.append(review_text.text)



NameError: name 'review' is not defined